# Create Dataframe from Wikipedia website # 

In [15]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
lista=pd.read_html(url)
np.shape(lista)

(3,)

In [30]:
df=lista[0]

In [31]:
Indexes=df[df['Borough'] == 'Not assigned'].index
df.drop(Indexes, inplace=True)
df.shape

(103, 3)

In [32]:
df2=df.groupby(['Postal code']).agg('sum')
df2['Neighborhood']=df2['Neighborhood'].str.replace('/', ',')
df2

,Borough,Neighborhood
Postal code,,
M1B,Scarborough,"Malvern , Rouge"
M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
M1E,Scarborough,"Guildwood , Morningside , West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."


In [33]:
df2.shape

(103, 2)

# Get coordinates using geocoder #

In [6]:
#pip install geocoder   ### Install geocoder
!conda install -c conda-forge geocoder -y
import geocoder

In [23]:
latitude=[]
longitude=[]
for PostalCode in df2.index:

# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(PostalCode))
      lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])


In [34]:
df2['Latitude']=latitude
df2['Longitude']=longitude

In [35]:
df2

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M1B,Scarborough,"Malvern , Rouge",43.808626,-79.189913
M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.785779,-79.157368
M1E,Scarborough,"Guildwood , Morningside , West Hill",43.765806,-79.185284
M1G,Scarborough,Woburn,43.771545,-79.218135
M1H,Scarborough,Cedarbrae,43.768791,-79.238813
...,...,...,...,...
M9N,York,Weston,43.705496,-79.520370
M9P,Etobicoke,Westmount,43.696296,-79.533126
M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.686887,-79.565507


# Exploring and Clustering#

In [42]:
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from geopy.geocoders import Nominatim


### Selecting boroughs containing the word toronto ###

In [40]:
Toronto_data = df2[df2['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data

,Borough,Neighborhood,Latitude,Longitude
0,East Toronto,The Beaches,43.678148,-79.295349
1,East Toronto,"The Danforth West , Riverdale",43.683424,-79.354564
2,East Toronto,"India Bazaar , The Beaches West",43.668291,-79.315578
3,East Toronto,Studio District,43.648000,-79.339260
4,Central Toronto,Lawrence Park,43.729455,-79.386415
5,Central Toronto,Davisville North,43.713171,-79.388870
6,Central Toronto,North Toronto West,43.714139,-79.406456
7,Central Toronto,Davisville,43.703327,-79.385649
8,Central Toronto,"Moore Park , Summerhill East",43.690328,-79.383522
9,Central Toronto,"Summerhill West , Rathnelly , South Hill , For...",43.686378,-79.402372


### Geographical coordinates of Toronto using Nominatim ###

In [43]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ontario are 43.6534817, -79.3839347.


In [44]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Explore Toronto in Foursquare ###

In [48]:
CLIENT_ID = '5325RSOXV3SY4GEG5IH5R4DL3ZEPY20FMM4VYIAN302XEW1N' # your Foursquare ID
CLIENT_SECRET = 'HSPMTGE4IHNX40VN0WZGI3LJLXVVKQYR4VXUOKSSUZAAVQNT' # your Foursquare Secret
VERSION = '20200415' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5325RSOXV3SY4GEG5IH5R4DL3ZEPY20FMM4VYIAN302XEW1N
CLIENT_SECRET:HSPMTGE4IHNX40VN0WZGI3LJLXVVKQYR4VXUOKSSUZAAVQNT


### Get nearby Venues function for all Neighborhoods (from Lab) ###

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst  Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park ,

### One hot encoding for the Venues Data frame ###

In [52]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronton_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.015625,0.000000,0.015625,...,0.000000,0.000000,0.00,0.00,0.015625,0.000000,0.000000,0.000000,0.00,0.015625
1,"Brockton , Parkdale Village , Exhibition Place",0.022222,0.000000,0.022222,0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.022222,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Business reply mail Processing CentrE,0.000000,0.030000,0.010000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.00,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.016667,0.016667,0.016667,0.00,0.000000
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,Church and Wellesley,0.000000,0.012195,0.000000,0.012195,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.012195
7,"Commerce Court , Victoria Hotel",0.000000,0.040000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.010000,0.000000,0.000000,0.010000,0.00,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.037037,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


### Top 10 Venues per Neighborhood ###

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Italian Restaurant,Breakfast Spot,Cocktail Bar,Hotel,Farmers Market,Bakery,Beer Bar,Cheese Shop
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Café,Gift Shop,Thrift / Vintage Store,Pizza Place,Diner,Accessories Store,North Indian Restaurant,Caribbean Restaurant,Chiropractor
2,Business reply mail Processing CentrE,Coffee Shop,Hotel,Restaurant,American Restaurant,Café,Italian Restaurant,Gastropub,Tea Room,Salon / Barbershop,Sandwich Place
3,"CN Tower , King and Spadina , Railway Lands , ...",Coffee Shop,Café,Restaurant,Park,French Restaurant,Lounge,Italian Restaurant,Bar,Speakeasy,Mexican Restaurant
4,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Breakfast Spot,Spa,Bubble Tea Shop,Electronics Store,Plaza,Clothing Store,Restaurant


### Clustering K-means ###

In [56]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 0, 3, 3, 3, 0])

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.678148,-79.295349,3.0,Health Food Store,Pub,Trail,Yoga Studio,Electronics Store,Food,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant
1,East Toronto,"The Danforth West , Riverdale",43.683424,-79.354564,0.0,Business Service,Park,Discount Store,Grocery Store,Bus Line,Falafel Restaurant,Food Truck,Food Court,Food & Drink Shop,Food
2,East Toronto,"India Bazaar , The Beaches West",43.668291,-79.315578,3.0,Fast Food Restaurant,Sandwich Place,Pub,Food & Drink Shop,Park,Burrito Place,Steakhouse,Sushi Restaurant,Movie Theater,Coffee Shop
3,East Toronto,Studio District,43.648000,-79.339260,0.0,Athletics & Sports,Government Building,Night Market,Business Service,Yoga Studio,Falafel Restaurant,Food Court,Food & Drink Shop,Food,Flower Shop
4,Central Toronto,Lawrence Park,43.729455,-79.386415,4.0,Bus Line,Swim School,Lawyer,Yoga Studio,Food Court,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop


### Folium Map ###

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[1],
        fill=True,
        fill_color=rainbow[1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters